In [284]:
import parse
from collections import deque
from copy import deepcopy
import itertools

def deck(n):
    return deque(range(n))

def part_1(shuffle, cards):
    # I'd prefer to use pyrsistent but it's incredibly slow here
    cards = deepcopy(cards)
    n = len(cards)

    for line in shuffle:
        line = line.strip()
        
        if line == 'deal into new stack':
            cards.reverse()
        elif r := parse.parse('cut {cut:d}', line):
            cards.rotate(-r['cut'])
        elif r := parse.parse('deal with increment {incr:d}', line):
            new_cards = [-1] * n
            for i in range(n):
                new_cards[(i * r['incr']) % n] = cards[i]
            cards = deque(new_cards)
        else:
            print('Unparsed line', line)
            
    return cards

In [285]:
test_deck = deck(10)

part_1(['deal into new stack'], test_deck)

deque([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

In [286]:
part_1(['cut 3'], test_deck)

deque([3, 4, 5, 6, 7, 8, 9, 0, 1, 2])

In [287]:
part_1(['cut -4'], test_deck)

deque([6, 7, 8, 9, 0, 1, 2, 3, 4, 5])

In [288]:
part_1(['deal with increment 3'], test_deck)

deque([0, 7, 4, 1, 8, 5, 2, 9, 6, 3])

In [289]:
test_shuffle = '''deal into new stack
cut -2
deal with increment 7
cut 8
cut -4
deal with increment 7
cut 3
deal with increment 9
deal with increment 3
cut -1'''.splitlines()

part_1(test_shuffle, test_deck)

deque([9, 2, 5, 8, 1, 4, 7, 0, 3, 6])

In [290]:
shuffle = open('input')
full_deck = deck(10007)

In [247]:
part_1(shuffle, full_deck)[2019]

9093

Wrong. Hmm.

(after a large amount of fruitless debugging)

In [291]:
part_1(shuffle, full_deck).index(2019)

4649

OFFFS.

## Part 2

This will require some thought.